In [29]:
%pip install -q -U google-genai google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [30]:
from google import genai
from google.genai import types
import os
import pandas as pd
import json
import time

In [31]:
API_KEY = os.getenv("GEMINI_API_KEY")
MODEL_NAME = "models/gemini-2.5-flash"

SYSTEM_INSTRUCTION = """# Vai trò: Chuyên gia Hiệu đính OCR Dược điển Cấp cao (ĐDVN V, Tập 2)

## Nhiệm vụ Cốt lõi
Thực hiện làm sạch, hiệu đính và phục hồi cấu trúc toàn diện cho văn bản OCR từ **Dược điển Việt Nam V, Tập 2**. Mục tiêu là sửa lỗi OCR chuyên sâu bằng **ngữ cảnh hai chiều** để đạt được **độ chính xác khoa học 100%** và tạo ra **văn bản đầu ra sạch tuyệt đối**.

## Yêu cầu Làm sạch (Ưu tiên Cao nhất)
1.  **Sửa lỗi OCR Chuyên sâu & Chính xác Khoa học**:
    * Sử dụng **ngữ cảnh hai chiều** để sửa 100% lỗi OCR (đánh máy, ký hiệu phức tạp/thuật ngữ dược/hóa học) (ví dụ: sửa `m[` thành `mL`, xử lý triệt để ký tự nhiễu như `[`, `]`, `}`, `\\`).
2.  **Loại bỏ Nhiễu Triệt để**: **Bắt buộc loại bỏ** Header, Footer (số trang/chú thích) và tất cả các ký tự/dấu ngoặc vô nghĩa, đặt sai vị trí (ví dụ: `[`, `]`, `{`, `}`, `\\`).
3.  **Phục hồi Cấu trúc Đoạn văn**: Nối các câu/ý bị ngắt dòng bất hợp lý (lỗi OCR) và đảm bảo chỉ giữ lại dấu xuống dòng ở cuối đoạn văn. Sửa lỗi nối liền các đoạn văn lẽ ra phải xuống dòng.

## Nguyên tắc & Định dạng
* **Ưu tiên cao nhất**: Đảm bảo **tính chính xác khoa học** và **độ sạch tuyệt đối** của văn bản (loại bỏ nhiễu và sửa lỗi OCR).
* **Không thêm/bớt thông tin** ngoài việc sửa lỗi.
* **Đầu ra**: Chỉ xuất ra nội dung văn bản đã được làm sạch, phục hồi cấu trúc và **không kèm bất kỳ giải thích hay ghi chú nào.**
"""

SYSTEM_INSTRUCTION_OBJECT = {
    "parts": [
        {
            "text": SYSTEM_INSTRUCTION
        }
    ]
}

client = genai.Client(api_key=API_KEY)

input_path = "../data/raw/raw_texts.txt"
chunks_export = "../data/raw/text_chunks.json"
batch_request = "../data/raw/batch_requests.jsonl"
output_path = "../data/raw/normalized_texts.csv"

chunk_size = 4000
overlap = 200

## Read data and split into chunks

In [32]:
with open(input_path, "r", encoding="utf-8") as f:
    raw_text = f.read()
    
chunks = []
chunk_id = 0
start = 0

while start < len(raw_text):
    end = min(start + chunk_size + overlap, len(raw_text))
    chunks.append({
        "chunk_id": chunk_id,
        "text": raw_text[start:end]
    })
    chunk_id += 1
    start += chunk_size


with open(chunks_export, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)
df = pd.DataFrame(chunks)
print("Total chunks:", df.shape[0])
df.head()

Total chunks: 271


,chunk_id,text
0,0,Dược ĐIÊN VIỆT NAM V\nQUI ĐỊNH CHUNG\n1. Tên c...
1,1,hất và lượng cân.\nKhái niệm “đâ cân trước” (đ...
2,2,ả theo\nphương pháp vi sinh vật để kết luận (t...
3,3,\n27. Hỗn hợp của các chất lỏng được ghi theo ...
4,4,c cổ truyền) phải đạt tiêu chuẩn Dược điển Việ...


## Create batch data

In [33]:
with open(batch_request, "w", encoding="utf-8") as f:
    for row in df.head(5).itertuples():  # Test with 5 samples only
        chunk_id = row.chunk_id
        text = row.text

        record = {
            "key": chunk_id,
            "request": {
                "contents": [
                    {
                        "parts": [{"text": text}]
                    }
                ],
                "system_instruction": SYSTEM_INSTRUCTION_OBJECT,
                "generation_config": {
                    "temperature": 0.2,
                    "top_p": 0.9,
                    "thinking_config": {"thinking_budget": 0}
                }
            }
        }
        f.write(json.dumps(record) + "\n")

## Submit batch data

In [ ]:
uploaded = client.files.upload(
    file=batch_request,
    config=types.UploadFileConfig(
        display_name="text-normalization-requests",
        mime_type="text/plain"
    )
)

batch_job = client.batches.create(
    model=MODEL_NAME,
    src=uploaded.name,
    config={"display_name": "text-normalization-batch-job"}
)
time.sleep(30) # Wait for batch job to start

## Polling for responses

In [37]:
while True:
    job = client.batches.get(name=batch_job.name)
    state = job.state.name
    print(state, end="\r")
    if state not in ("JOB_STATE_RUNNING", "JOB_STATE_PENDING"):
        print(f"[INFO] {job.state.name}")
        break
    time.sleep(5)
    
if state != "JOB_STATE_SUCCEEDED":
    print("[ERROR] Job failed", job.state)
else:
    print(job.name)

[INFO] JOB_STATE_SUCCEEDED
batches/fp0t208369eysmwsnaubssyd1bdn0nobfgdb


In [38]:
records_list = []

result_file = job.dest.file_name
raw = client.files.download(file=result_file).decode('utf-8')
records = [json.loads(line) for line in raw.splitlines()]

## Save results

In [39]:
temp_dict = []
errors = []
for rec in records:
    try:
        key = rec.get("key", "")
        
        result = rec["response"]["candidates"][0]["content"]["parts"][0]["text"]
        
        temp_dict.append({
            "chunk_id": key,
            "processed_text": result
        })
    except Exception as e:
        print(e)
        errors.append((key, str(e)))

In [40]:
df_result = pd.DataFrame(temp_dict)
df_merged = df.head(5).merge(df_result, on='chunk_id', how='left')
df_merged

,chunk_id,text,processed_text
0,0,Dược ĐIÊN VIỆT NAM V\nQUI ĐỊNH CHUNG\n1. Tên c...,DƯỢC ĐIỂN VIỆT NAM V\nQUY ĐỊNH CHUNG\n1. Tên c...
1,1,hất và lượng cân.\nKhái niệm “đâ cân trước” (đ...,hất và lượng cân.\nKhái niệm “đã cân trước” (đ...
2,2,ả theo\nphương pháp vi sinh vật để kết luận (t...,theo phương pháp vi sinh vật để kết luận (trừ ...
3,3,\n27. Hỗn hợp của các chất lỏng được ghi theo ...,27. Hỗn hợp của các chất lỏng được ghi theo ký...
4,4,c cổ truyền) phải đạt tiêu chuẩn Dược điển Việ...,cổ truyền) phải đạt tiêu chuẩn Dược điển Việt ...


In [41]:
df_merged.to_csv(output_path, index=False)